## Model-based collaborative filtering system

### SVD Matrix Factorization

### Theory
+ A = u x S x V
+ A : original matrix (utility matrix)
+ u : left orthogonal matrix -holds important,non redundant information about users
+ v : rigth orthogonal matrix - holds important, non redundant information on items
+ S : diagonal matrix, contains all of the information about the decomposition processes performed during the compression 

In [1]:
# import libs
import numpy as np
import pandas as pd

import sklearn
from sklearn.decomposition import TruncatedSVD

+ The MovieLens dataset was collected by the GroupLens Research Project at the University of Minnesota. You can download the dataset for this demostration at the following URL: https://grouplens.org/datasets/movielens/100k/

### Prepare the  data

### Explain the dataset : 
+ Movie recommendation using model-based recommendation system
+ Movie and user rating

In [3]:
columns= ['user_id','item_id','rating','timestamp']
frame = pd.read_csv('ml-100k/u.data', sep='\t', names=columns)
frame.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
columns = ['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
          'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('ml-100k/u.item',sep='|',names= columns,encoding='latin-1')

In [6]:
movie_names = movies[['item_id', 'movie title']]
movie_names.head()

,item_id,movie title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


### merge 2 datasets

In [7]:
combined_movies_data = pd.merge(frame,movie_names,on='item_id')
combined_movies_data.head()

,user_id,item_id,rating,timestamp,movie title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


### check rating from item_id

In [9]:
combined_movies_data.groupby('item_id')['rating'].count().sort_values(ascending=False).head()

item_id
50     583
258    509
100    508
181    507
294    485
Name: rating, dtype: int64

#### Filter combined datasets 

In [10]:
filter = combined_movies_data['item_id'] == 50

In [11]:
combined_movies_data[filter]['movie title'].unique()

array([u'Star Wars (1977)'], dtype=object)

## Building a Utility Matrix

In [12]:
# rating crosstab
rating_crosstab = combined_movies_data.pivot_table(values='rating', index='user_id', columns='movie title', fill_value=0)
rating_crosstab.head()

movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,2,5,0,0,3,4,0,0,...,0,0,0,5,3,0,0,0,4,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,2,0,0,0,0,4,0,0,...,0,0,0,4,0,0,0,0,4,0


### Transpose the matrix

In [14]:
rating_crosstab.shape

(943, 1664)

In [15]:
X = rating_crosstab.T
X.shape

(1664, 943)

### Decompose the matrix


In [19]:
# SVD declare
SVD = TruncatedSVD(n_components=12,random_state = 17)

In [20]:
resultant_matrix = SVD.fit_transform(X)

In [21]:
resultant_matrix.shape

(1664, 12)

## Generate a correlation matrix

In [22]:
corr_mat = np.corrcoef(resultant_matrix)

In [23]:
corr_mat.shape

(1664, 1664)

## Isolation Star Wars from the Correlation Matrix

In [24]:
movie_names = rating_crosstab.columns
movie_list = list(movie_names)

star_wars = movie_list.index('Star Wars (1977)')
print(star_wars)

1398


In [39]:
corr_star_wars = corr_mat[1398]

In [40]:
print(corr_star_wars)

[ 0.36854685  0.42380505  0.59468319 ...,  0.33301726  0.64177278
  0.26805079]


In [45]:
corr_star_wars = corr_mat[star_wars]
corr_star_wars.shape

(1664,)

## Recommend a Highly Correlated Movie

In [46]:
list(movie_names[(corr_star_wars<1.0) & (corr_star_wars>0.9)])

[u'Die Hard (1988)',
 u'Empire Strikes Back, The (1980)',
 u'Fugitive, The (1993)',
 u'Raiders of the Lost Ark (1981)',
 u'Return of the Jedi (1983)',
 u'Star Wars (1977)',
 u'Terminator 2: Judgment Day (1991)',
 u'Terminator, The (1984)',
 u'Toy Story (1995)']

In [47]:
list(movie_names[(corr_star_wars<1.0) & (corr_star_wars>0.98)])

[u'Return of the Jedi (1983)', u'Star Wars (1977)']

In [49]:
list(movie_names[(corr_star_wars<1.0) & (corr_star_wars>0.98)])

[u'Return of the Jedi (1983)', u'Star Wars (1977)']